In [1]:
import random
import timeit

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm import tqdm

In [2]:
class ClsNet(nn.Module):
    def __init__(self, num_classes=7, **kwargs):
        super(ClsNet, self).__init__()
        self.num_classes = num_classes

        self.classifier = nn.Linear(294912, 7, bias=False)

    def forward(self, x):
        bs = x.size(0)
        out = x.flatten(1)
        out = self.classifier(out)
        out = F.softmax(out, dim=1)
        return out


model = ClsNet().cuda()
img = torch.zeros(10, 3, 384, 256).cuda()
out = model(img)
print(out.size())

torch.Size([10, 7])


In [3]:
torch.save(model, "cls.pt")

In [4]:
model_loaded = torch.load("/home/hy/project/reid_task/analysis/cls.pt").cpu()
out = model_loaded(img.cpu())
print(out.size())

torch.Size([10, 7])


In [5]:
from PIL import Image

transforms1 = T.Compose(
    [
        T.Resize((384, 256), interpolation=3),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

img_obtained = np.random.random((64, 64, 3)).astype(np.uint8)
img_obtained = Image.fromarray(img_obtained)
img_obtained = transforms1(img_obtained).unsqueeze(0)

print(img_obtained.shape)
model_loaded = torch.load("/home/hy/project/reid_task/analysis/cls.pt").cpu()
out = model_loaded(img_obtained.cpu()).squeeze(0)
print(out.size())

torch.Size([1, 3, 384, 256])
torch.Size([7])
